Source https://favtutor.com/blogs/dynamic-programming

Disclaimer: I wrote my own code - syntax, organization, standarization over the problems solved. -

# The first problem: Knapsack Problem

Given a bag with capacity to carry at most weight $W$ and a group of $n$ items (for our purposes, they are ordered) with weights $w_1, w_2, \ldots, w_n$ and values $v_1, v_2, \ldots, v_n$. Find the maximum value possible of carrying a subset of these objects in the knapsack, and thus respecting its weight capacity.

## Why Dynamic Programming?
1. The problem can be decomposed in subproblems that would lead to the solution of this problem. It requires enough memory, but it speeds the process.
2. An alternative solution could be using genetic algorithms, but they do not lead in general to the optimal solution.

## The recursion.
The recursion of using subproblems consists in forming the matrix $M_{n \times W} = (m_{i,w})$ where $m_{i,w}$ is the maximum value that could be carried using only the first $i$ objects (that is why we have given some order, but the solution is independent of the order) and up to capacity is $w$.

The border cases are:
$$m_{i, 0} = 0$$
$$m_{0, w \geq 0} = 0$$

The recurssion is given by:
$$m_{i, w} = m_{i-1, w} \Leftarrow w_i > w$$
$$m_{i, w} = \max([m_{i-1, w - w_i} + v_i, m_{i-1, w}]) \Leftarrow w_i \leq w$$


In [10]:
# Exercise 1
capacity = 11
weights = [1, 2, 5, 6, 7]
profits = [1, 6, 18, 22, 28]

# is the problem clear?
assert len(weights) == len(profits)

# Derived Quantities
n_objects = len(weights)

# Form the sub problems matrix holder with empty values.
matrix = [
        [None for _ in range(capacity + 1)]
        for _ in range(n_objects + 1)]

# Apply initial conditions
for object_count in range(n_objects + 1):
    matrix[object_count][0] = 0
    
for sub_capacity in range(capacity + 1):
    matrix[0][sub_capacity] = 0

# Having a down-to-top approach to fill the subproblem matrix:
for object_count in range(1, n_objects + 1):
    for sub_capacity in range(1, capacity + 1):
        if weights[object_count - 1] > sub_capacity:
            # Cannot take the object, so the previous solution without the new object stands.
            matrix[object_count][sub_capacity] = matrix[object_count - 1][sub_capacity]
        else:
            # Gets the value if we use the object.
            matrix[object_count][sub_capacity] = max([
                # value with new object.
                ( 
                    matrix[object_count - 1][sub_capacity - weights[object_count - 1]] + 
                    profits[object_count - 1]
                ),
                # value without new object.
                matrix[object_count - 1][sub_capacity]
            ])
            
print(f"""
    The maximum value, one can carry in the knapsack
    with capacity W = 11 is: {matrix[n_objects][capacity]}
""")


    The maximum value, one can carry in the knapsack
    with capacity W = 11 is: 40



# Resulting Matrix and Answer
The answer or maximum profit that could be carried is 40,
which can be read from the lowest right corner in the filled matrix.


o\w| 0| 1| 2| 3| 4| 5| 6| 7| 8| 9|10|11
---|--|--|--|--|--|--|--|--|--|--|--|-- 
  0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0
  1| 0| 1| 1| 1| 1| 1| 1| 1| 1| 1| 1| 1
  2| 0| 1| 6| 7| 7| 7| 7| 7| 7| 7| 7| 7
  3| 0| 1| 6| 7| 7|18|19|24|25|25|25|25
  4| 0| 1| 6| 7| 7|18|22|24|28|29|29|40
  5| 0| 1| 6| 7| 7|18|22|28|29|34|35|40

But how can we read the object selected to achieve that maximum? Notice first that the solution for the 5 objects and capacity 11 is: $M_{o=5, w=11} = 40$. Notice that $M_{o=4, w=11} = 40$, which means we don't take the $5$-th object: $[-, -, -, -, 0]$ (here the 0 in this $5$-th position of this list represents that we don't take the $5$-th element.

o\w| 0| 1| 2| 3| 4| 5| 6| 7| 8| 9|10|11
---|--|--|--|--|--|--|--|--|--|--|--|-- 
  0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0
  1| 0| 1| 1| 1| 1| 1| 1| 1| 1| 1| 1| 1
  2| 0| 1| 6| 7| 7| 7| 7| 7| 7| 7| 7| 7
  3| 0| 1| 6| 7| 7|18|19|24|25|25|25|25
  4| 0| 1| 6| 7| 7|18|22|24|28|29|29|40


The weight of the $4$-th element is $6$, so to obtain $M_{o=4, w=11}$ we considered: $M_{o=3, w=5} + v_{o=4} = 18 + 22 = 40$, which is attained by taking the $4$-th element, or $M_{o=3, w=11} = 25$, which is attained by not taking the $4$-th element. Clearly the first one is bigger, so we took the $4$-th element: $[-, -, -, 1, 0]$. This could have been seen by the fact that $M_{o=3, w=11} < M_{o=4, w=11}$.

o\w| 0| 1| 2| 3| 4| 5
---|--|--|--|--|--|--
  0| 0| 0| 0| 0| 0| 0
  1| 0| 1| 1| 1| 1| 1
  2| 0| 1| 6| 7| 7| 7
  3| 0| 1| 6| 7| 7|18

Just as before, since $M_{o=3, w=5} > M_{o=3, w=5}$, then we know that we used the $3$rd object: $[-, -, 1, 1, 0]$, and that $M_{o=3, w=5} = M_{o=2, w=0} + 18 = 0 + 18 = 18$ because of the initial conditions. Clearly this implies that the selection is $[0. 0, 1, 1, 0]$.

# Formal Code

But, do we need to solve the entire matrix? Can we solve it from bottom to top? This could be achieved by using a recursive approach.


In [31]:
def get_max_of_backpack(
    capacity,
    weights,
    values
):
    """
    Gets the maximum value obtain by carring a subset of the objects in knapsack.
    
    Arguments:
        capacity (int): capacity of the knapsack in the problem units (e.g. lb)
        weights (list[int]): weights of the objects in the same units as the capacity.
        values (list[float]): profit or value of each object.
    Raises:
        AssertionError: if the lengths of the weights and the values is different.
    Returns:
        tuple[int, list]:
        - (float): maximum profit or value that could be carried in the knapsack.
        - (list): matrix of the solution of the subproblems associated.
    """
    
    assert len(weights) == len(values)
    n_objects = len(weights)
    matrix = [
        [None for _ in range(capacity + 1)]
        for _ in range(n_objects + 1)]

    def solve_subproblem(
        i_objects,
        w_capacity
    ):  
        """
        This subfunction is meant to be called recursively to solve
        the recursive equation of the subproblems.
        
        It is written inside the main function to avoid creating a function that
        would require many arguments, since we use the original arguments
        of the outer function, plus the current state arguments described below.
        
        Arguments:
             i_objects (int): number of objects used in the subproblem.
             w_capacity (int): the maximum capacity used in the subproblem.
        
        Returns:
             (float): maximum profit of the subproblem.
        """
        
        if w_capacity == 0 or i_objects == 0:
            return 0

        if matrix[i_objects][w_capacity] is not None:
            return matrix[i_objects][w_capacity]

        if weights[i_objects - 1] <= w_capacity:
            matrix[i_objects][w_capacity] = max([
                solve_subproblem(
                    i_objects - 1,
                    w_capacity),
                solve_subproblem(
                    i_objects - 1,
                    w_capacity - weights[i_objects - 1],
                ) + values[i_objects - 1]
            ])  
            return matrix[i_objects][w_capacity]
        else:
            matrix[i_objects][w_capacity] = solve_subproblem(
                i_objects - 1,
                w_capacity)
            return matrix[i_objects][w_capacity]
    
 
    result = solve_subproblem(n_objects, capacity)

    return result, matrix    

capacity = 11
weights = [1, 2, 5, 6, 7]
profits = [1, 6, 18, 22, 28]

result, matrix = get_max_of_backpack(
    capacity=capacity, weights=weights, values=profits)

print("The max profit that could be carried:", result)

The max profit that could be carried: 40


In [21]:
import pandas as pd
# Printing the matrix:
df = pd.DataFrame(
    matrix, 
    columns=[f'w{str(ind).zfill(2)}' for ind in range(capacity + 1)],
    index=[f'o{str(ind).zfill(2)}' for ind in range(n_objects + 1)]
).fillna("")
df

,w00,w01,w02,w03,w04,w05,w06,w07,w08,w09,w10,w11
o00,,,,,,,,,,,,
o01,,,1.0,1.0,1.0,1.0,1.0,,,1.0,,1.0
o02,,,,,7.0,7.0,7.0,,,,,7.0
o03,,,,,7.0,18.0,,,,,,25.0
o04,,,,,7.0,,,,,,,40.0
o05,,,,,,,,,,,,40.0


# Compare the bottom-to-top approach.
In this small problem, we can see that we did not fill most of the matrix of subproblems. I suspect in a larger setting, this could speed the computation. Still, we are missing a subprocess to obtain the list of objects selected.

In [30]:
# An initial solution
def get_objects_selected(subcapacity, weights, matrix):
    object_index = len(weights)
    selection = [0 for _ in range(len(weights))]
    while object_index > 0 and subcapacity > 0:
        if matrix[object_index - 1][subcapacity] != matrix[object_index][subcapacity]:
            selection[object_index - 1] = 1
            subcapacity -= weights[object_index - 1]
        object_index -= 1
    return selection
    
selection = get_objects_selected(
    subcapacity=11,  # we want to know the result for the entire problem.
    weights=weights, 
    matrix=matrix)

print("The selection is:", selection)

for ind, chosen in enumerate(selection):
    if chosen == 1:
        print(f"The {ind + 1}-th element was selected to be carried in the knapsack.")



The selection is: [0, 0, 1, 1, 0]
The 3-th element was selected to be carried in the knapsack.
The 4-th element was selected to be carried in the knapsack.


# Subset Sum Problem
Given an array of non-negative integers $[a_1, a_2, \ldots, a_n]$ (not necessarily sorted, and target value $T$. Find if there is a subset, whose sum is equal to the target.

The recursion here to solve is: Consider the matrix $M_{(n+1) \times (T+1)} = (m_{i,t})$ where $m_{i,t}$ is whether there is a subset from the first $i$ elements of the original array whose sum is $t$.

$$m_{0, t > 0} = \textrm{False}$$
$$m_{i, 0} = \textrm{True}$$

$$m_{i, t} = m_{i-1, t} \Leftarrow a_i > t$$
$$m_{i, t} = m_{i-1, t} \textrm{ or } m_{i-1, t - a_i} \Leftarrow a_i \leq t$$ 

In [ ]:
def is_there_a_solution(my_array, target):
    
    n_objects = len(my_array)
    matrix = [
        [None for _ in range(target + 1)]
        for _ in range(len(my_array) + 1)]
    
    def solve_subproblem(i_objects, subtarget):
        if subtarget == 0:
            return True
        
        if i_objects == 0 and subtarget > 0:
            return False
        
        if matrix[i_objects][subtarget] is not None:
            return matrix[i_objects][subtarget]
        
        if my_array[i_objects - 1] > subtarget:
            matrix[i_objects][subtarget] = solve_subproblem(i_objects - 1, subtarget)
            return matrix[i_objects][subtarget]
        
        else: 
            matrix[i_objects][subtarget] = (
                solve_subproblem(i_objects - 1, subtarget) or
                solve_subproblem(i_objects - 1, subtarget - my_array[i_objects - 1])
            )
            return matrix[i_objects][subtarget]
        
    result = solve_subproblem(n_objects, target)
    return result, matrix
                
my_array = [0, 3, 2, 7, 1]
target = 6
result, matrix = is_there_a_solution(my_array=my_array, target=target)
print(result)
matrix

# Longest Common Subsequence

**Problem**: Given two strings $s_1$ and $s_2$, of respective lengths $l_1$ and $l_2$. Find the size of the longest common subsequence $s_1$ and $s_2$. Recall that a subsequence is more general than a substring because the elements don't need to be consequtive.

**Recurrence**: The matrix $M_{l_1 \times l_2} = (m_{i, j})$ where $m_{i, j}$ is the size of the longest common subsequence from the substrings $t_1$ and $t_2$ formed by taking the first $i$ characters of $s_1$ and the first $j$ characters of $t_2$.

If $s_1[i] == s_2[j]$, then 
$$m_{i,j} = m_{i-1, j-1} + 1$$
Else:
$$m_{i,j} = \max([m_{i-1, j}, m_{i, j-1}])$$

In [33]:
def measure_longest_subsequence(string1, string2):
    """
    Finds the longest subsequence common to two strings.
    Recall that a subsequence is not the same as a substring, because
    the characters don't need to be consecutive.
    
    Arguments:
        string1 (str): first string.
        string2 (str): second string.
    Returns:
        (tuple)
        - (int): the length of the longest common subsequence.
        - (list): list of list of the solution of the subproblems required to
           obtain the solution of the problem.
    """
    len1 = len(string1)
    len2 = len(string2)
    matrix = [
        [None for _ in range(len2 + 1)]
        for _ in range(len1 + 2)]
    
    def solve_subproblem(sublen1, sublen2):
        if sublen1 == 0 or sublen2 == 0:
            return 0
        
        if matrix[sublen1][sublen2] is not None:
            return matrix[sublen1][sublen2]
        
        if string1[sublen1 - 1] == string2[sublen2 - 1]:
            matrix[sublen1][sublen2] = solve_subproblem(sublen1 - 1, sublen2 - 1) + 1

        else:
            matrix[sublen1][sublen2] = max([
                solve_subproblem(sublen1 - 1, sublen2),
                solve_subproblem(sublen1, sublen2 - 1)
            ])
        return matrix[sublen1][sublen2]
    
    result = solve_subproblem(len1, len2)
    return result, matrix

s1 = 'abbacdcba'
s2 = 'bcdbbcaa'
result, matrix = measure_longest_subsequence(s1, s2)
print(result)
matrix

5


[[None, None, None, None, None, None, None, None, None],
 [None, 0, 0, 0, 0, 0, 0, None, None],
 [None, 1, 1, 1, 1, 1, 1, None, None],
 [None, 1, 1, 1, 2, 2, 2, None, None],
 [None, 1, 1, 1, 2, 2, None, 3, None],
 [None, 1, 2, 2, 2, 2, 3, 3, None],
 [None, 1, None, 3, 3, 3, 3, 3, None],
 [None, None, 2, 3, 3, None, 4, 4, None],
 [None, None, None, None, None, 4, 4, 4, None],
 [None, None, None, None, None, None, None, None, 5],
 [None, None, None, None, None, None, None, None, None]]

In [35]:
# An alternative way of printing the matrix would be to use the pretty print library.
# But based on the indentation, our approach above of using a frame is still better.
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(matrix)

[   [None, None, None, None, None, None, None, None, None],
    [None, 0, 0, 0, 0, 0, 0, None, None],
    [None, 1, 1, 1, 1, 1, 1, None, None],
    [None, 1, 1, 1, 2, 2, 2, None, None],
    [None, 1, 1, 1, 2, 2, None, 3, None],
    [None, 1, 2, 2, 2, 2, 3, 3, None],
    [None, 1, None, 3, 3, 3, 3, 3, None],
    [None, None, 2, 3, 3, None, 4, 4, None],
    [None, None, None, None, None, 4, 4, 4, None],
    [None, None, None, None, None, None, None, None, 5],
    [None, None, None, None, None, None, None, None, None]]
